In [1]:
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as tf
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [29]:
train_list_df = pd.read_csv('train_val_list.txt', header=None, names=['Image Index'])
test_list_df = pd.read_csv('test_list.txt', header=None, names=['Image Index'])
xray_train_df = pd.read_csv('BBox_List_2017.csv')
xray_test_df = pd.read_csv('Data_entry_2017_v2020.csv')

In [30]:
train_list_df.shape, test_list_df.shape

((86524, 1), (25596, 1))

In [31]:
train_list_df.head(10)

,Image Index
0,00000001_000.png
1,00000001_001.png
2,00000001_002.png
3,00000002_000.png
4,00000004_000.png
5,00000005_000.png
6,00000005_001.png
7,00000005_002.png
8,00000005_003.png
9,00000005_004.png


In [32]:
test_list_df.sample(10)

,Image Index
3685,00005962_007.png
13775,00016587_083.png
25471,00030641_000.png
13132,00016009_025.png
24737,00030111_002.png
4719,00007670_027.png
2893,00004808_066.png
5013,00008269_005.png
21525,00027470_000.png
22528,00028536_000.png


In [33]:
xray_test_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [34]:
xray_test_df.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')

In [35]:
def firstchoice(sr):
    return sr.split('|')[0]

In [37]:
xray_test_df["Finding Labels"] = xray_test_df["Finding Labels"].apply(firstchoice)

In [41]:
xray_test_df.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')

In [43]:
xray_test_df = xray_test_df.rename(columns={'OriginalImage[Width': 'w', 'Height]': 'h', 'OriginalImagePixelSpacing[x': 'x', 'y]':'y'})

In [44]:
xray_test_df = xray_test_df.drop(['Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position'], axis=1)

In [54]:
xray_test_df = xray_test_df.rename(columns={'Finding Labels': 'Finding Label'})

In [55]:
xray_test_df.sample(10)

,Image Index,Finding Label,w,h,x,y
101402,00026943_000.png,No Finding,2992,2991,0.143,0.143
66850,00016521_003.png,Consolidation,2500,2048,0.168,0.168
32988,00008638_003.png,No Finding,2048,2500,0.168,0.168
59350,00014674_021.png,No Finding,2500,2048,0.168,0.168
30726,00008009_036.png,No Finding,2500,2048,0.168,0.168
3741,00001019_000.png,No Finding,2992,2991,0.143,0.143
94515,00024276_000.png,No Finding,2992,2991,0.143,0.143
7334,00001939_000.png,No Finding,3056,2544,0.139,0.139
79167,00019434_005.png,Atelectasis,3056,2544,0.139,0.139
80965,00019894_010.png,Effusion,2500,2048,0.168,0.168


In [56]:
xray_train_df.sample(10)

,Image Index,Finding Label,x,y,w,h
897,00025252_054.png,Pneumothorax,659.342222,720.535712,144.497778,225.280000
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153
134,00008716_000.png,Atelectasis,707.589418,709.756614,151.703704,104.025397
315,00021703_001.png,Effusion,563.768889,511.184601,208.213333,155.875556
536,00021926_007.png,Infiltrate,323.995767,273.066667,151.703704,190.713228
695,00020274_021.png,Nodule,159.288889,437.773545,100.774603,68.266667
93,00016009_008.png,Atelectasis,572.139683,576.474074,154.954497,249.227513
370,00002583_014.png,Effusion,608.982011,397.680423,318.577778,407.432804
454,00019399_010.png,Effusion,783.441270,602.480423,133.282540,249.227513
674,00014116_009.png,Nodule,696.753439,319.661376,54.179894,57.430688


In [47]:
xray_train_df = xray_train_df.rename(columns={'Bbox [x': 'x', 'h]': 'h'})

In [48]:
xray_train_df = xray_train_df.dropna(axis=1)

In [49]:
xray_train_df.sample(10)

,Image Index,Finding Label,x,y,w,h
944,00028861_009.png,Pneumothorax,581.892063,47.678307,291.487831,300.156614
317,00018102_001.png,Effusion,107.520000,588.553490,38.684444,53.475556
70,00030106_008.png,Atelectasis,225.084746,517.731081,229.966102,269.016949
134,00008716_000.png,Atelectasis,707.589418,709.756614,151.703704,104.025397
697,00026132_016.png,Nodule,701.087831,482.201058,80.186243,74.768254
160,00020810_003.png,Atelectasis,159.288889,715.174603,258.979894,113.777778
902,00017137_016.png,Pneumothorax,170.097778,147.095712,287.857778,475.591111
513,00001039_005.png,Infiltrate,588.393651,275.233862,210.217989,279.568254
478,00029861_013.png,Infiltrate,193.991111,185.780156,608.711111,501.760000
859,00004808_090.png,Pneumonia,135.449735,237.307937,289.320635,363.005291


In [57]:
xray_df = pd.concat([xray_test_df, xray_train_df])

/Users/senhmo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [58]:
xray_df.sample(10)

,Finding Label,Image Index,h,w,x,y
76665,No Finding,00018840_027.png,2544.0,3056.0,0.139000,0.139000
94635,No Finding,00024374_000.png,2048.0,2500.0,0.168000,0.168000
19555,Infiltration,00005200_004.png,2048.0,2500.0,0.171000,0.171000
9357,Infiltration,00002437_012.png,2048.0,2500.0,0.168000,0.168000
35825,No Finding,00009437_004.png,2021.0,2021.0,0.194311,0.194311
91133,Effusion,00022720_003.png,2991.0,2992.0,0.143000,0.143000
29251,No Finding,00007621_008.png,2048.0,2500.0,0.168000,0.168000
20502,No Finding,00005493_000.png,2048.0,2500.0,0.171000,0.171000
69367,No Finding,00017120_000.png,2753.0,2670.0,0.143000,0.143000
65248,No Finding,00016106_000.png,2991.0,2992.0,0.143000,0.143000


In [59]:
def my_hot_encoding(dataframe, feature_list_to_encode):
    encoding_dict = {}
    for each_feature in feature_list_to_encode:
        feature_dict = {}
        for index_type, each_type in enumerate(dataframe[each_feature].unique()):
            feature_dict[each_type] = index_type
        encoding_dict[each_feature] = feature_dict
    for each_feature in feature_list_to_encode:
        dataframe[each_feature] = dataframe[each_feature].map(lambda x: encoding_dict[each_feature][x])
    dataframe[each_feature].astype(np.int64)
    return encoding_dict  

In [60]:
disease_encoding = my_hot_encoding(xray_df, ['Finding Label'])

In [62]:
disease_encoding

{'Finding Label': {'Cardiomegaly': 0,
  'No Finding': 1,
  'Hernia': 2,
  'Mass': 3,
  'Infiltration': 4,
  'Effusion': 5,
  'Nodule': 6,
  'Emphysema': 7,
  'Atelectasis': 8,
  'Pneumothorax': 9,
  'Pleural_Thickening': 10,
  'Fibrosis': 11,
  'Consolidation': 12,
  'Edema': 13,
  'Pneumonia': 14,
  'Infiltrate': 15}}

In [61]:
xray_df.sample(10)

,Finding Label,Image Index,h,w,x,y
52078,10,00013128_023.png,2048.0,2500.0,0.168,0.168
4829,1,00001298_005.png,2048.0,2500.0,0.168,0.168
54095,4,00013616_035.png,2048.0,2500.0,0.168,0.168
19686,8,00005226_000.png,2500.0,2048.0,0.168,0.168
72277,12,00017800_005.png,2048.0,2500.0,0.168,0.168
13881,8,00003577_001.png,2853.0,2982.0,0.143,0.143
79512,8,00019530_016.png,2508.0,3056.0,0.139,0.139
26618,1,00006973_010.png,2048.0,2500.0,0.168,0.168
56811,1,00014125_041.png,2544.0,3056.0,0.139,0.139
50203,1,00012697_006.png,2991.0,2992.0,0.143,0.143


In [63]:
xray_df['Image Index'].isin(train_list_df['Image Index']).value_counts()

True     86524
False    26580
Name: Image Index, dtype: int64

In [64]:
xray_df['Image Index'].isin(test_list_df['Image Index']).value_counts()

False    86524
True     26580
Name: Image Index, dtype: int64

In [65]:
train_df = xray_df[xray_df['Image Index'].isin(train_list_df['Image Index'])]

In [67]:
test_df = xray_df[xray_df['Image Index'].isin(test_list_df['Image Index'])]

In [ ]:
# test_df = test_df.drop(['No Finding', 'Finding Label'], axis=1)

In [68]:
test_df.to_csv('test_df2.csv', index=False)

In [69]:
train_df.to_csv('train_df2.csv', index=False)